# Практическое задание 4

1. Попробуйте улучшить точность распознавания образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложите анализ с описанием того, что улучшает работу нейронной сети, а что ухудшает

2. Опишите в анализе, какие изменения необходимо было бы внести в получившуюся нейронную сеть, если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET

In [1]:
# import numpy as np
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D


# tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import os
import matplotlib.pyplot as plt

In [4]:
# установка параметров нейросети
batch_size = 128
num_classes = 10
epochs = 5
data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [5]:
# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

170498071/170498071 [==============================] - 184s 1us/step
x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [6]:
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=0.5)

In [7]:
# plt.imshow(x_train[0])

In [8]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [9]:
# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [10]:
# запуск data augmentation через fit_generator
model.fit(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 1/5
391/391 [==============================] - 59s 149ms/step - loss: 2.0481 - accuracy: 0.2434 - val_loss: 1.7744 - val_accuracy: 0.3856
Epoch 2/5
391/391 [==============================] - 60s 154ms/step - loss: 1.7603 - accuracy: 0.3616 - val_loss: 1.5848 - val_accuracy: 0.4288
Epoch 3/5
391/391 [==============================] - 64s 164ms/step - loss: 1.6384 - accuracy: 0.4045 - val_loss: 1.4934 - val_accuracy: 0.4653
Epoch 4/5
391/391 [==============================] - 65s 167ms/step - loss: 1.5628 - accuracy: 0.4309 - val_loss: 1.4204 - val_accuracy: 0.4881
Epoch 5/5
391/391 [==============================] - 64s 163ms/step - loss: 1.5055 - accuracy: 0.4528 - val_loss: 1.3954 - val_accuracy: 0.4958


In [19]:
# # сохранение модели и весов
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# model_path = os.path.join(save_dir, model_name)
# model.save(model_path)
# print('сохранить обученную модель как %s ' % model_path)


сохранить обученную модель как C:\Users\Pirozhok\Documents\geekbrains\введение в нейронные сети\saved_models\keras_cifar10_trained_model.h5 


In [11]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 2s 7ms/step - loss: 1.3954 - accuracy: 0.4958
Test loss: 1.395448088645935
Test accuracy: 0.4957999885082245


Уведичим размер ядра и шаг в пулинговом слое, уменьшим dropout

In [12]:
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (5, 5), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.1))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [13]:
# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [14]:
# запуск data augmentation через fit_generator
model.fit(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 1/5
391/391 [==============================] - 103s 261ms/step - loss: 1.8921 - accuracy: 0.3059 - val_loss: 1.6342 - val_accuracy: 0.4038
Epoch 2/5
391/391 [==============================] - 105s 267ms/step - loss: 1.6518 - accuracy: 0.3953 - val_loss: 1.5178 - val_accuracy: 0.4451
Epoch 3/5
391/391 [==============================] - 111s 284ms/step - loss: 1.5440 - accuracy: 0.4387 - val_loss: 1.4534 - val_accuracy: 0.4629
Epoch 4/5
391/391 [==============================] - 108s 277ms/step - loss: 1.4772 - accuracy: 0.4647 - val_loss: 1.4276 - val_accuracy: 0.4894
Epoch 5/5
391/391 [==============================] - 110s 282ms/step - loss: 1.4124 - accuracy: 0.4919 - val_loss: 1.2885 - val_accuracy: 0.5307


In [15]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 3s 10ms/step - loss: 1.2885 - accuracy: 0.5307
Test loss: 1.2885239124298096
Test accuracy: 0.5307000279426575


Качество улучшилось, но обучение было долгим

Увеличим шаг, уменьшим droput

In [19]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), strides=(2,2), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), strides=(2,2), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [20]:
# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [21]:
# запуск data augmentation через fit_generator
model.fit(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 1/5
391/391 [==============================] - 17s 42ms/step - loss: 2.1619 - accuracy: 0.1790 - val_loss: 1.9830 - val_accuracy: 0.2690
Epoch 2/5
391/391 [==============================] - 19s 49ms/step - loss: 1.9861 - accuracy: 0.2619 - val_loss: 1.8717 - val_accuracy: 0.3363
Epoch 3/5
391/391 [==============================] - 20s 51ms/step - loss: 1.8952 - accuracy: 0.3004 - val_loss: 1.7866 - val_accuracy: 0.3595
Epoch 4/5
391/391 [==============================] - 20s 51ms/step - loss: 1.8173 - accuracy: 0.3312 - val_loss: 1.7176 - val_accuracy: 0.3863
Epoch 5/5
391/391 [==============================] - 20s 50ms/step - loss: 1.7630 - accuracy: 0.3507 - val_loss: 1.6356 - val_accuracy: 0.4075


In [22]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 3ms/step - loss: 1.6356 - accuracy: 0.4075
Test loss: 1.635570764541626
Test accuracy: 0.4074999988079071


Простое увеличение шага привело к ухудшению результата, но обучение проходило быстрее. Поэтому увеличим шаг только на пуллинговом слое

In [23]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.1))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [24]:
# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [25]:
# запуск data augmentation через fit_generator
model.fit(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)

Epoch 1/5
391/391 [==============================] - 57s 144ms/step - loss: 1.9532 - accuracy: 0.2848 - val_loss: 1.7231 - val_accuracy: 0.3922
Epoch 2/5
391/391 [==============================] - 58s 149ms/step - loss: 1.7271 - accuracy: 0.3752 - val_loss: 1.5789 - val_accuracy: 0.4255
Epoch 3/5
391/391 [==============================] - 58s 147ms/step - loss: 1.6210 - accuracy: 0.4116 - val_loss: 1.4738 - val_accuracy: 0.4666
Epoch 4/5
391/391 [==============================] - 58s 147ms/step - loss: 1.5433 - accuracy: 0.4377 - val_loss: 1.4100 - val_accuracy: 0.4952
Epoch 5/5
391/391 [==============================] - 57s 147ms/step - loss: 1.4879 - accuracy: 0.4605 - val_loss: 1.3483 - val_accuracy: 0.5169


In [26]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 3s 8ms/step - loss: 1.3483 - accuracy: 0.5169
Test loss: 1.3483290672302246
Test accuracy: 0.5169000029563904
